In [2]:
import numpy as np
import os 
import re
from scipy import stats
import pandas as pd

"""Dieses Sktipt liegt separat von der Analyse der Decoderzeiten, da ich neue Messdaten 
   mit zusätzlich den Beam-Search-Zeiten aufnahm"""


def get_file_lines(model, cd : int):
    dir_ = ['Messreihe1/', 'Messreihe2/'][cd] #cd = Cuda-Device = Graphikkarte für Testung (0 = TITAN V, 1 = TITAN RTX)
    lines = np.array([])
    for path, __, files in os.walk(dir_): 
        for file in files:
            if model == file[:-4]:
                lines = np.append(lines, open(path + file, 'r').readlines())
    return lines

models = ['lstm_bs64','lstm_bs32','lstm_bs8', 'lstm_bs8_cd0', 'gru_bs64', 'gru_bs32', 'gru_bs8', 'gru_bs8_cd0']

data = {model : {'Messreihe1' : get_file_lines(model, 0), 'Messreihe2' : get_file_lines(model, 1)} for model in models}


In [3]:
def get_gflp_times_per_batch(lines : np.array): #Zeit für Funktion CopyNetSeq2Seq._gather_final_log_probs
    re1 = re.compile('g_f_logprobs :')  
    re2 = re.compile('Batch [1-6] Test Time =')
    
    bli = batched_line_indexes = np.array(np.where([bool(re.search(re2, el)) for el in lines])).flatten()
    out = {}
    for i in range(6):
        if i != 0:
            l_ = lines[bli[i-1]:bli[i]]
        else:
            l_ = lines[:bli[i]] 
        indexes_gflp = np.array(np.where([bool(re.search(re1, el)) for el in l_])).flatten()
        tgather_logprobs = np.array([float(line[15:-1]) for line in l_[indexes_gflp]]) 
        out.update({str(i+1) : tgather_logprobs})
    return out
                                                          
def analyse_gflp(model, batches=range(6)):
    l1, l2 = data[model]['Messreihe1'], data[model]['Messreihe2']
    data_dict = {'Messreihe1' : get_gflp_times_per_batch(data[model]['Messreihe1']),\
                 'Messreihe2' : get_gflp_times_per_batch(data[model]['Messreihe2'])}

    cols = pd.MultiIndex.from_arrays([['1']*6+['2']*6,[str(i+1) for i in batches]*2], names=['Messreihe','Testbatch'])
    rows = ['mean', 'std', 'max', 'min', 'sum', 'count']
    funcs = [np.mean, np.std, np.max, np.min, np.sum, len]
    df_dict1 = {str(key) : {func_key : func(data_dict['Messreihe1'][str(key)]) \
                               for (func_key, func) in zip(rows, funcs) } for key in range(1,7)}
    df_dict2 = {str(key) : {func_key : func(data_dict['Messreihe2'][str(key)]) \
                               for (func_key, func) in zip(rows, funcs) } for key in range(1,7)}
    
    print(f'{model}')
    df1 = pd.DataFrame.from_dict(df_dict1)
    df2 = pd.DataFrame.from_dict(df_dict2)
    df = pd.concat([df1,df2], axis=1)
    df.columns = cols
    df = df.round(3)
    display(df)

In [4]:
print('Zeiten für Funktion gather_final_log_probs: \n')
for k in data.keys():
    analyse_gflp(k)

Zeiten für Funktion gather_final_log_probs: 

lstm_bs64


Messreihe        1                                                     2  \
Testbatch        1        2        3        4        5        6        1   
mean         0.201    0.172    0.153    0.154    0.168    0.073    0.208   
std          0.108    0.088    0.080    0.079    0.083    0.044    0.113   
max          1.114    0.317    0.293    0.290    0.307    0.146    1.018   
min          0.062    0.038    0.036    0.040    0.039    0.006    0.063   
sum        179.884   85.911   76.395   77.158   83.854   25.381  186.323   
count      897.000  500.000  500.000  500.000  500.000  347.000  897.000   

Messreihe                                               
Testbatch        2        3        4        5        6  
mean         0.212    0.189    0.191    0.207    0.095  
std          0.108    0.098    0.096    0.102    0.057  
max          0.389    0.359    0.352    0.373    0.184  
min          0.048    0.044    0.050    0.048    0.007  
sum        105.949   94.293   95.343  103.406   32.824  
count      500.000  500.000  500.000  500.000  347.000

lstm_bs32


Messreihe        1                                                     2  \
Testbatch        1        2        3        4        5        6        1   
mean         0.178    0.173    0.165    0.165    0.171    0.102    0.173   
std          0.082    0.089    0.088    0.086    0.088    0.062    0.095   
max          0.755    0.323    0.314    0.309    0.316    0.200    1.037   
min          0.064    0.039    0.036    0.041    0.039    0.006    0.058   
sum        124.661   86.733   82.737   82.698   85.695   47.142  122.801   
count      700.000  500.000  500.000  500.000  500.000  461.000  710.000   

Messreihe                                              
Testbatch        2        3        4        5       6  
mean         0.123    0.111    0.103    0.131   0.008  
std          0.058    0.048    0.045    0.062   0.002  
max          0.230    0.170    0.176    0.252   0.013  
min          0.036    0.033    0.037    0.036   0.005  
sum         55.455   44.011   39.110   60.465   0.182  
count      450.000  398.000  380.000  460.000  23.000

lstm_bs8


...

Messreihe        1                                                    2  \
Testbatch        1        2        3        4        5       6        1   
mean         0.196    0.113    0.112    0.118    0.123   0.013    0.195   
std          0.118    0.042    0.043    0.050    0.046   0.004    0.111   
max          1.312    0.188    0.188    0.211    0.179   0.022    1.136   
min          0.062    0.046    0.043    0.048    0.047   0.007    0.059   
sum        153.793   36.699   43.160   39.015   38.108   0.419  152.886   
count      783.000  325.000  384.000  331.000  310.000  33.000  783.000   

Messreihe                                              
Testbatch        2        3        4        5       6  
mean         0.117    0.116    0.122    0.127   0.013  
std          0.044    0.044    0.052    0.048   0.005  
max          0.195    0.194    0.217    0.185   0.023  
min          0.048    0.045    0.050    0.049   0.007  
sum         38.039   44.725   40.405   39.457   0.439  
count      325.000  384.000  331.000  310.000  33.000

gru_bs8


Messreihe        1                                                     2  \
Testbatch        1        2        3        4        5        6        1   
mean         0.173    0.160    0.158    0.162    0.161    0.102    0.173   
std          0.080    0.082    0.082    0.082    0.082    0.062    0.078   
max          0.934    0.294    0.292    0.294    0.294    0.203    0.780   
min          0.064    0.036    0.033    0.037    0.036    0.005    0.064   
sum        121.180   80.212   78.858   81.035   80.555   51.128  121.049   
count      700.000  500.000  500.000  500.000  500.000  500.000  700.000   

Messreihe                                               
Testbatch        2        3        4        5        6  
mean         0.161    0.159    0.163    0.162    0.103  
std          0.083    0.083    0.083    0.083    0.062  
max          0.297    0.295    0.298    0.296    0.202  
min          0.036    0.033    0.038    0.037    0.005  
sum         80.700   79.292   81.519   80.993   51.251  
count      500.000  500.000  500.000  500.000  500.000

gru_bs8_cd0


Messreihe        1                                                    2  \
Testbatch        1        2        3        4        5       6        1   
mean         0.197    0.093    0.093    0.097    0.101   0.010    0.197   
std          0.116    0.035    0.035    0.041    0.038   0.003    0.120   
max          1.112    0.157    0.156    0.174    0.148   0.017    1.124   
min          0.064    0.038    0.036    0.040    0.039   0.006    0.064   
sum        154.094   30.313   35.633   32.193   31.455   0.336  154.231   
count      783.000  325.000  384.000  331.000  310.000  33.000  783.000   

Messreihe                                              
Testbatch        2        3        4        5       6  
mean         0.093    0.093    0.097    0.101   0.010  
std          0.035    0.035    0.041    0.038   0.003  
max          0.157    0.156    0.176    0.148   0.017  
min          0.038    0.035    0.040    0.039   0.006  
sum         30.268   35.585   32.186   31.408   0.336  
count      325.000  384.000  331.000  310.000  33.000

In [15]:
def get_beam_search_times_per_batch(lines : np.array):
    re1 = re.compile('beam_search_time: ')
    re2 = re.compile('Batch [1-6] Test Time =')
    out = {}
    bli = batched_line_indexes = np.array(np.where([bool(re.search(re2, el)) for el in lines])).flatten()
    for i in range(6):
        if i != 0:
            l_ = lines[bli[i-1]:bli[i]]
        else:
            l_ = lines[:bli[i]] 
        indexes = np.array(np.where([bool(re.search(re1, el)) for el in l_])).flatten()
        bs_times = np.array([float(line[18:-3]) for line in l_[indexes]])
        out.update({str(i+1) : bs_times})
    return out # {Batch : array(times)}

def get_gflp_times_per_extraction(lines : np.array): #Zeit pro Beam-Search bzw. pro generierten Satz einer iteration
    re1 = re.compile('g_f_logprobs :')
    re2 = re.compile('Batch [1-6] Test Time =')
    re3 = re.compile('beam_search_time: ')
    
    li = line_indexes = np.array(np.where([bool(re.search(re2, el)) for el in lines])).flatten()
    out = {}
    for i in range(6):
        out.update({str(i+1) : None})
    for i in range(6):
        if i != 0:
            l_ = lines[li[i-1]:li[i]]
        else:
            l_ = lines[:li[i]] 
        bsli = beam_search_line_indexes = np.array(np.where([bool(re.search(re3, el)) for el in l_])).flatten()
        out2 = {}
        for j in range(len(bsli)):
            if j != 0:
                l__ = l_[bsli[j-1]:bsli[j]]
            else:
                l__ = l_[:bsli[j]] 
            indexes_gflp = np.array(np.where([bool(re.search(re1, el)) for el in l__])).flatten()
            tgather_logprobs = np.array([float(line[14:-3]) for line in l__[indexes_gflp]])
            out2.update({str(j+1) : tgather_logprobs}) 
        out[str(i+1)] = out2
    return out #{Batch : extraktionsnr : gflp_werte}
               # Batch = range(6), extraktionsnr in [1-18], len(gflp_werte) = 7-50

def get_gflp_time_percentages(model):   
    perc1, perc2 = np.array([]), np.array([])
    l1, l2 = data[model]['Messreihe1'], data[model]['Messreihe2']
    #{Messreihe : Batch : extraktionsnr : array(gflp_werte)}
    data_dict = {'Messreihe1' : get_gflp_times_per_extraction(l1),\
                 'Messreihe2' : get_gflp_times_per_extraction(l2)}
    #{Messreihe : Batch : array(bstimes)} 
    bs_time_dict = {'Messreihe1' : get_beam_search_times_per_batch(l1),\
                    'Messreihe2' : get_beam_search_times_per_batch(l2)}
    #{Batch : np.array(perc)}
    #Formel : 100*summe(gflp)/bstime = Anteil
    perc_dict1 = {str(i+1) :\
                  100*np.array([np.sum(data_dict['Messreihe1'][str(i+1)][j])\
                                for j in data_dict['Messreihe1'][str(i+1)].keys()])\
                  /bs_time_dict['Messreihe1'][str(i+1)]\
                  for i in range(6)} 
    #{Batch : np.array(perc)}
    perc_dict2 = {str(i+1) :\
                  100*np.array([np.sum(data_dict['Messreihe2'][str(i+1)][j])\
                                for j in data_dict['Messreihe2'][str(i+1)].keys()])\
                  /bs_time_dict['Messreihe2'][str(i+1)]\
                  for i in range(6)}
        
    return perc_dict1, perc_dict2

def analyse(models):
    mean_vals=[]
    valcounts, valsum = 0,0
    print('Durchschnittlicher Zeitanteil der Funktion gather_final_log_probs je Modell: \n')
    for model in models:
        perc_dict1, perc_dict2 = get_gflp_time_percentages(model)
        
        perc_dict1.pop('1') #--> der erste Batch wird ausgelassen, da große Abweichungen dadurch Existieren, weil 
        perc_dict2.pop('1') #2 Testungen von IMoJIE gleichzeitig ausgeführt wurden
        
        #Da nun jeder Batch maximal 10 Extraktionen hat aber die Anzahl ...
        #an Extraktionen unterschiedlich ist,müssen die Werte aufgefüllt werden
        
        for i in range(1,6):
            perc_dict1[str(i+1)] = np.append(perc_dict1[str(i+1)],np.array([np.nan]*(10-len(perc_dict1[str(i+1)]))))
            perc_dict2[str(i+1)] = np.append(perc_dict2[str(i+1)],np.array([np.nan]*(10-len(perc_dict2[str(i+1)]))))
        #print(perc_dict1)
        df1 = pd.DataFrame.from_dict(perc_dict1)
        #print(perc_dict2)
        df2 = pd.DataFrame.from_dict(perc_dict2)
        df = pd.concat([df1,df2], axis=1)
        #df.indexes = range(1,10)

        df.index = pd.MultiIndex.from_arrays([range(1,11)], names=['Extraktion'])        
        cols = pd.MultiIndex.from_arrays([[str(i+1) for i in range(1,6)]*2], names=['Testbatch'])
        df.columns = cols 
        
        print(f'{model}:\nDurchschnittlicher Anteil je Modell: {np.mean(df.mean())}')
        
        
        mean_vals.append(np.mean(df.mean())) 
        valsum += np.sum(df.sum())
        valcounts += np.sum(df.count(numeric_only=True))
        df = df.round(5)
        display(df)
    print(f'durchschnittswerte der durchschnittlichen Modell-Anteile: {np.mean(mean_vals)}')
    print(f'{model}:\nGesamtdurchschnitt aller einzelwerte: {valsum/valcounts}')
    


In [16]:
print(models, '\n')
analyse(models)

['lstm_bs64', 'lstm_bs32', 'lstm_bs8', 'lstm_bs8_cd0', 'gru_bs64', 'gru_bs32', 'gru_bs8', 'gru_bs8_cd0'] 

Durchschnittlicher Zeitanteil der Funktion gather_final_log_probs je Modell: 

lstm_bs64:
Durchschnittlicher Anteil je Modell: 97.49318805093984


Testbatch,2,3,4,5,6,2,3,4,5,6
Extraktion,,,,,,,,,,
1,96.03029,95.72576,96.21896,96.12519,79.13181,95.96176,95.65509,96.14091,96.02614,79.16820
2,97.65267,97.37783,97.41409,97.67728,86.65272,97.61251,97.31633,97.36843,97.63075,86.87888
3,98.32736,98.19680,98.09973,98.33462,89.58535,98.30641,98.17011,98.06761,98.31356,89.77352
4,98.69184,98.47205,98.56245,98.70436,92.71405,98.69794,98.46123,98.55287,98.69684,92.80921
5,98.91017,98.72333,98.84184,98.92604,94.01870,98.93594,98.75246,98.84884,98.93610,94.09488
6,99.08419,98.95237,98.95611,99.07386,96.85623,99.10762,98.97797,98.99296,99.10869,96.89673
7,99.20315,99.10758,99.09738,99.17699,97.86247,99.23259,99.13401,99.12827,99.21239,97.89265
8,99.29362,99.22057,99.21555,99.25746,98.37827,99.31999,99.25130,99.24803,99.27557,98.40462
9,99.34903,99.30283,99.30045,99.33566,98.69482,99.39018,99.33334,99.31702,99.36470,98.71412


lstm_bs32:
Durchschnittlicher Anteil je Modell: 96.8090860582047


Testbatch,2,3,4,5,6,2,3,4,5,6
Extraktion,,,,,,,,,,
1,96.06278,95.75220,96.22616,96.11441,78.99897,96.02411,95.72865,96.19795,96.09973,78.74895
2,97.67170,97.39034,97.40065,97.59302,86.54338,97.64844,97.34880,97.37190,97.61465,86.35633
3,98.32462,98.17474,98.14946,98.29072,95.58172,98.32737,98.07798,98.09390,98.30503,90.14286
4,98.66375,98.58861,98.58668,98.65461,97.34416,98.41344,98.49179,98.30914,98.52022,NaN
5,98.91997,98.84249,98.86266,98.91341,98.09937,98.70808,98.76310,98.56403,98.80074,NaN
6,99.07638,99.03060,99.02847,99.05935,98.51560,98.86819,98.97577,98.78937,98.96224,NaN
7,99.19588,99.16538,99.14079,99.18537,98.78421,99.02724,99.01408,98.89400,99.03071,NaN
8,99.27769,99.26189,99.26469,99.28129,98.95995,99.14066,99.06753,99.00909,99.16107,NaN
9,99.36151,99.34268,99.34073,99.35585,99.09551,99.20674,99.06873,99.10070,99.25625,NaN


lstm_bs8:
Durchschnittlicher Anteil je Modell: 95.34060322691137


...

gru_bs8:
Durchschnittlicher Anteil je Modell: 98.05030170104098


Testbatch,2,3,4,5,6,2,3,4,5,6
Extraktion,,,,,,,,,,
1,95.63480,95.24691,95.76913,95.71061,78.71883,95.66297,95.26684,95.77568,95.71929,78.54666
2,97.50313,97.39569,97.56273,97.58325,94.95449,97.54006,97.39702,97.56632,97.60593,94.95045
3,98.23702,98.19611,98.27154,98.29834,97.13753,98.24561,98.20111,98.28005,98.29514,97.13543
4,98.62301,98.59170,98.64179,98.66328,97.99139,98.62495,98.59624,98.64177,98.66800,97.98883
5,98.86736,98.82472,98.87463,98.89620,98.45506,98.87080,98.82807,98.88159,98.89775,98.45055
6,99.03212,99.01989,99.03794,99.05626,98.74576,99.03768,99.02167,99.04104,99.05883,98.73934
7,99.15842,99.14845,99.16257,99.17594,98.93576,99.15782,99.14872,99.16646,99.17911,98.92137
8,99.23128,99.24041,99.25168,99.26566,99.07676,99.24040,99.24640,99.25424,99.27123,99.07287
9,99.32767,99.32100,99.33017,99.34322,99.18640,99.33049,99.32283,99.33289,99.34265,99.18191


gru_bs8_cd0:
Durchschnittlicher Anteil je Modell: 96.0691174763568


Testbatch,2,3,4,5,6,2,3,4,5,6
Extraktion,,,,,,,,,,
1,96.17217,95.87447,96.33341,96.20255,79.42076,96.15354,95.84307,96.28753,96.19072,79.43376
2,97.72740,97.43683,97.62336,97.74689,86.87886,97.71363,97.42556,97.60985,97.73192,86.90178
3,98.17480,97.88096,97.95780,98.39149,89.85351,98.15737,97.87571,97.95767,98.38208,89.84856
4,98.17440,98.23315,98.22637,98.54964,92.39937,98.15960,98.22274,98.22358,98.55200,92.34303
5,98.48557,98.26324,98.52195,98.71377,NaN,98.48531,98.24991,98.51766,98.70922,NaN
6,98.70816,98.59953,98.72713,98.83481,NaN,98.67443,98.59135,98.72127,98.82709,NaN
7,98.74794,98.78587,98.87582,98.93577,NaN,98.72960,98.77788,98.87317,98.93096,NaN
8,98.84324,98.84759,98.94128,98.93602,NaN,98.84128,98.84078,98.95079,98.93031,NaN
9,98.93352,98.97895,99.01668,98.93059,NaN,98.93566,98.97081,99.01217,98.92617,NaN


durchschnittswerte der durchschnittlichen Modell-Anteile: 96.69292172204469
gru_bs8_cd0:
Gesamtdurchschnitt aller einzelwerte: 97.62779378775825
